In [1]:
import pandas as pd
import numpy
import scipy.stats as sts
from collections import defaultdict

In [2]:
housing_data = pd.read_csv('/Users/aaronzeanah/Documents/GitHub/etl_project/RDC_Inventory_Hotness_Metrics_County_History.csv')
housing_data['year'] = housing_data['month_date_yyyymm'].astype(str)
housing_data['year'] = housing_data['year'].str[0:4]
housing_data.drop('month_date_yyyymm', axis=1, inplace=True)
housing_data['year'] = housing_data['year'].astype(int)
housing_data.sort_values(by=['cbsa_code'], inplace=True, ascending=[True])
housing_data.head()

,county_fips,county_name,cbsa_code,cbsa_title,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,...,median_days_on_market_vs_us,ldpviews_per_property_mm,ldpviews_per_property_yy,ldpviews_per_property_vs_us,median_listing_price,median_listing_price_mm,median_listing_price_yy,median_listing_price_vs_us,quality_flag,year
44722,53027,"grays harbor, wa",10140,"aberdeen, wa",745,722,NaN,NaN,31.831832,38.738739,...,6.5,NaN,NaN,0.7447,179500.0000,NaN,NaN,0.6955,0,2016
2789,53027,"grays harbor, wa",10140,"aberdeen, wa",745,789,76.0,94.0,27.177177,31.431431,...,13.0,0.0977,0.5532,0.7766,249950.0000,0.0018,0.0647,0.8061,0,2020
32578,53027,"grays harbor, wa",10140,"aberdeen, wa",745,578,8.0,144.0,43.993994,61.061061,...,-6.5,-0.0204,0.3714,0.7619,199950.0000,0.0000,0.1139,0.7165,0,2017
10627,53027,"grays harbor, wa",10140,"aberdeen, wa",745,627,63.0,-39.0,41.791792,55.655656,...,-5.0,-0.0256,-0.0656,0.8507,245050.0000,-0.0255,0.1041,0.7538,0,2019
13868,53027,"grays harbor, wa",10140,"aberdeen, wa",745,868,15.0,-382.0,20.920921,27.327327,...,27.0,0.1125,-0.2078,0.6421,239278.5714,0.0193,0.0879,0.7788,0,2019


In [3]:

# housing_data.sort_values(by=['cbsa_code'], inplace=True, ascending=[True])
# housing_data.groupby('year')
# housing_data.set_index=('year')
housing_data.columns

Index(['county_fips', 'county_name', 'cbsa_code', 'cbsa_title',
       'nielsen_hh_rank', 'hotness_rank', 'hotness_rank_mm', 'hotness_rank_yy',
       'hotness_score', 'supply_score', 'demand_score',
       'median_days_on_market', 'median_days_on_market_mm',
       'median_days_on_market_mm_day', 'median_days_on_market_yy',
       'median_days_on_market_yy_day', 'median_days_on_market_vs_us',
       'ldpviews_per_property_mm', 'ldpviews_per_property_yy',
       'ldpviews_per_property_vs_us', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'median_listing_price_vs_us', 'quality_flag', 'year'],
      dtype='object')

In [4]:
# good_movies_df = movie_file_df.loc[movie_file_df["IMDB"] > 7, [
#     "FILM", "IMDB", "IMDB_user_vote_count"]]
# good_movies_df.head()

housing_2016 = housing_data.loc[housing_data["year"] == 2016].set_index("cbsa_code")
housing_2017 = housing_data.loc[housing_data["year"] == 2017].set_index("cbsa_code")
housing_2018 = housing_data.loc[housing_data["year"] == 2018].set_index("cbsa_code")
housing_2019 = housing_data.loc[housing_data["year"] == 2019].set_index("cbsa_code")
housing_2020 = housing_data.loc[housing_data["year"] == 2020].set_index("cbsa_code")
housing_2016.head()

,county_fips,county_name,cbsa_title,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,demand_score,...,median_days_on_market_vs_us,ldpviews_per_property_mm,ldpviews_per_property_yy,ldpviews_per_property_vs_us,median_listing_price,median_listing_price_mm,median_listing_price_yy,median_listing_price_vs_us,quality_flag,year
cbsa_code,,,,,,,,,,,,,,,,,,,,,
10140,53027,"grays harbor, wa","aberdeen, wa",745,722,NaN,NaN,31.831832,38.738739,24.924925,...,6.5,NaN,NaN,0.7447,179500.0,NaN,NaN,0.6955,0,2016
10140,53027,"grays harbor, wa","aberdeen, wa",745,719,NaN,NaN,31.381381,40.140140,22.622623,...,5.0,NaN,NaN,0.7083,175050.0,NaN,NaN,0.6757,0,2016
10140,53027,"grays harbor, wa","aberdeen, wa",745,847,NaN,NaN,19.069069,15.515516,22.622623,...,29.5,NaN,NaN,0.6905,175049.5,NaN,NaN,0.6864,0,2016
10140,53027,"grays harbor, wa","aberdeen, wa",745,857,NaN,NaN,18.818819,15.015015,22.622623,...,29.0,NaN,NaN,0.6667,169950.0,NaN,NaN,0.6799,0,2016
10140,53027,"grays harbor, wa","aberdeen, wa",745,806,NaN,NaN,22.272272,24.824825,19.719720,...,16.0,NaN,NaN,0.6512,179950.0,NaN,NaN,0.6985,0,2016


In [6]:
housing_data_2016 = housing_2016.groupby('cbsa_code')
housing_data_2016 = housing_data_2016.mean()
housing_data_2016.to_csv("housing_files/housing_data_2016.csv", index=True, header=True)

# housing_data_2017 = housing_2017.groupby('cbsa_code')
# housing_data_2017 = housing_data_2017.mean()
# housing_data_2017.to_csv("housing_files/housing_data_2017.csv", index=True, header=True)

# housing_data_2018 = housing_2018.groupby('cbsa_code')
# housing_data_2018 = housing_data_2018.mean()
# housing_data_2018.to_csv("housing_files/housing_data_2018.csv", index=True, header=True)

# housing_data_2019 = housing_2019.groupby('cbsa_code')
# housing_data_2019 = housing_data_2019.mean()
# housing_data_2019.to_csv("housing_files/housing_data_2019.csv", index=True, header=True)

# housing_data_2020 = housing_2020.groupby('cbsa_code')
# housing_data_2020 = housing_data_2020.mean()
# housing_data_2020.to_csv("housing_files/housing_data_2020.csv", index=True, header=True)

housing_data_2016.head(10)

,county_fips,nielsen_hh_rank,hotness_rank,hotness_rank_mm,hotness_rank_yy,hotness_score,supply_score,demand_score,median_days_on_market,median_days_on_market_mm,...,median_days_on_market_vs_us,ldpviews_per_property_mm,ldpviews_per_property_yy,ldpviews_per_property_vs_us,median_listing_price,median_listing_price_mm,median_listing_price_yy,median_listing_price_vs_us,quality_flag,year
cbsa_code,,,,,,,,,,,,,,,,,,,,,
10140,53027.0,745.00,778.833333,NaN,NaN,25.734067,28.695362,22.772773,94.333333,NaN,...,15.416667,NaN,NaN,0.698117,177174.750000,NaN,NaN,0.692017,0.0,2016.0
10180,48441.0,458.00,332.333333,NaN,NaN,64.556223,56.956957,72.155489,73.833333,NaN,...,-5.083333,NaN,NaN,1.236667,173140.833333,NaN,NaN,0.676217,0.0,2016.0
10300,26091.0,609.00,183.333333,NaN,NaN,76.726727,65.031698,88.421755,69.500000,NaN,...,-9.416667,NaN,NaN,1.600817,152241.666667,NaN,NaN,0.594667,0.0,2016.0
10420,39143.0,254.00,237.750000,NaN,NaN,71.901068,63.004671,80.797464,70.458333,NaN,...,-8.458333,NaN,NaN,1.391108,146426.041667,NaN,NaN,0.572017,0.0,2016.0
10460,35035.0,808.00,982.166667,NaN,NaN,4.412746,5.555556,3.269937,122.583333,NaN,...,43.666667,NaN,NaN,0.376500,166966.666667,NaN,NaN,0.652133,0.0,2016.0
10500,13095.0,632.00,989.833333,NaN,NaN,2.694361,5.055055,0.333667,126.333333,NaN,...,47.416667,NaN,NaN,0.273383,88650.000000,NaN,NaN,0.346183,0.0,2016.0
10540,41043.0,508.00,458.666667,NaN,NaN,53.912246,72.856190,34.968302,64.916667,NaN,...,-14.000000,NaN,NaN,0.815567,243307.500000,NaN,NaN,0.950700,0.0,2016.0
10580,36067.0,315.25,704.416667,NaN,NaN,32.225976,36.707541,27.744411,87.041667,NaN,...,8.125000,NaN,NaN,0.740629,233258.104167,NaN,NaN,0.911204,0.0,2016.0
10620,37167.0,848.00,927.166667,NaN,NaN,11.077745,1.885219,20.270270,141.416667,NaN,...,62.500000,NaN,NaN,0.671833,275079.333333,NaN,NaN,1.074617,0.0,2016.0
